In [ ]:
import os, sys, time, re, json
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
import random

In [ ]:
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)
print(client.request('vget /unrealcv/status'))

In [ ]:
def parse_color(color_str):
    # Use regular expression to find all numbers
    numbers = re.findall(r'\d+', color_str)

    # Convert the list of strings to a list of integers
    array = list(map(int, numbers))
    return (array[0], array[1], array[2])

def parse_rotation(rotation_str):
    # Use regular expression to find all numbers including negatives and decimals
    numbers = re.findall(r'-?\d+\.\d+', rotation_str)

    # Convert the list of strings to a list of floats
    array = list(map(float, numbers))
    return (array[0], array[1], array[2])

In [ ]:
# Get the list of all objects in the scene
scene_objects = client.request('vget /objects').split(' ')
scene_objects.pop()

person_colors = set()
for obj in scene_objects:
    color_str = client.request('vget /object/%s/color' % obj)
    color = parse_color(color_str)
    if (obj.startswith('SM_Child') or obj.startswith('SM_Couple') or obj.startswith('SM_Man') or obj.startswith('SM_Trio') or obj.startswith('SM_Woman')) and not obj.startswith('SM_Manhole'):
        if not color in person_colors:
            person_colors.add(color)
        else:
            print('Duplicate color:', color)
            sys.exit(-1)
    elif color in person_colors:
        print('Duplicate color:', color)
        sys.exit(-1)

In [ ]:
def annotate_persons(object_mask):
    


In [ ]:
# get the cameras in the scene:
cameras = len(client.request('vget /cameras').split(' ')) - 1

rotation_factor = 0.8
randoms = random.sample(range(100000), 10000)

for i in range(1, cameras): # skip the first camera
    # get camera rotation
    rot_str = client.request('vget /camera/%d/rotation' % i)
    rot = parse_rotation(rot_str)
    print(rot)
    rotation_angle = 0
    while rotation_angle < 360:
        # set camera rotation
        client.request('vset /camera/%d/rotation %f %f %f' % (i, rot[0], rot[1] + rotation_angle, rot[2]))
        rotation_angle += rotation_factor
        lit_img = client.request('vget /camera/%d/lit png' % i)
        img = PIL.Image.open(BytesIO(lit_img))
        rand = randoms.pop()
        img.save("./data/img_%d.png" % rand)
        